In [1]:
import os
import random
import math

import numpy as np
import pandas as pd
import pdb
from collections import OrderedDict
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import collections  as mc
matplotlib.rcParams['figure.figsize'] = [6, 6]
matplotlib.rcParams['figure.dpi'] = 200

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms

from data_helper import UnlabeledDataset, LabeledDataset
from helper import draw_box

# random.seed(0)
# np.random.seed(0)
# torch.manual_seed(0);

# All the images are saved in image_folder
# All the labels are saved in the annotation_csv file
image_folder = '/scratch/brs426/data'
annotation_csv = '/scratch/brs426/data/annotation.csv'

# You shouldn't change the unlabeled_scene_index
# The first 106 scenes are unlabeled
unlabeled_scene_index = np.arange(106)
# The scenes from 106 - 133 are labeled
# You should devide the labeled_scene_index into two subsets (training and validation)
train_labeled_scene_index = np.arange(106, 132)
val_labeled_scene_index = np.arange(132, 134)
test_labeled_scene_index = np.arange(132, 134)

print(train_labeled_scene_index[-1])
from helper import compute_ats_bounding_boxes, compute_ts_road_map

131


In [2]:
def round_up(x):
    return int(math.ceil(x / 50.0)) * 50

def round_down(x):
    return round_up(x) - 50

In [3]:
class_label = 0
class_dict = dict()
reverse_class_dict = []
for i in range(0, 800, 50):
    for j in range(0, 800, 50):
        class_dict[(i, j)] = class_label
        class_label += 1
        reverse_class_dict.append((i, j))

In [4]:
class_dict

{(0, 0): 0,
 (0, 50): 1,
 (0, 100): 2,
 (0, 150): 3,
 (0, 200): 4,
 (0, 250): 5,
 (0, 300): 6,
 (0, 350): 7,
 (0, 400): 8,
 (0, 450): 9,
 (0, 500): 10,
 (0, 550): 11,
 (0, 600): 12,
 (0, 650): 13,
 (0, 700): 14,
 (0, 750): 15,
 (50, 0): 16,
 (50, 50): 17,
 (50, 100): 18,
 (50, 150): 19,
 (50, 200): 20,
 (50, 250): 21,
 (50, 300): 22,
 (50, 350): 23,
 (50, 400): 24,
 (50, 450): 25,
 (50, 500): 26,
 (50, 550): 27,
 (50, 600): 28,
 (50, 650): 29,
 (50, 700): 30,
 (50, 750): 31,
 (100, 0): 32,
 (100, 50): 33,
 (100, 100): 34,
 (100, 150): 35,
 (100, 200): 36,
 (100, 250): 37,
 (100, 300): 38,
 (100, 350): 39,
 (100, 400): 40,
 (100, 450): 41,
 (100, 500): 42,
 (100, 550): 43,
 (100, 600): 44,
 (100, 650): 45,
 (100, 700): 46,
 (100, 750): 47,
 (150, 0): 48,
 (150, 50): 49,
 (150, 100): 50,
 (150, 150): 51,
 (150, 200): 52,
 (150, 250): 53,
 (150, 300): 54,
 (150, 350): 55,
 (150, 400): 56,
 (150, 450): 57,
 (150, 500): 58,
 (150, 550): 59,
 (150, 600): 60,
 (150, 650): 61,
 (150, 700): 62,

In [5]:
reverse_class_dict[7]

(0, 350)

In [4]:
def collate_fn(batch):
    BLOCK_SIZE = 10
    images = []
    target = []
    road_maps = []
    road_bins = []
    bbs = []
    target_counts = []
    for x in batch:
        
        grid = []
        # Get road_image and cast it to float
        road_image = torch.as_tensor(x[2])
        road_maps.append(road_image)
        road_image = road_image.float()
        
        # Split up into blocks and assign pixel value for block
        for x_ in range(0, 800, BLOCK_SIZE):
            for y in range(0, 800, BLOCK_SIZE):
                block = road_image[x_:x_+BLOCK_SIZE, y:y+BLOCK_SIZE]
                score = torch.sum(block).item()
                # If more than have the pixels are 1, classify as road
                if score > (BLOCK_SIZE**2) / 2:
                    grid.append(1.0)
                else:
                    grid.append(0.0)
                
        road_bins.append(torch.Tensor(grid))
        
        # Collect six images for this sample. 
        six_images = []
        for i in range(6):
            six_images.append(torch.Tensor(x[0][i]))
        
        
        # target
        bb_tens = x[1]['bounding_box']
        current_bbs = []
        bins = np.zeros(256)
        counts = np.zeros(90)
        count = 0
        
        for i, corners in enumerate(bb_tens):
#             if x[1]['category'][i] not in [1, 3, 6, 8]:
            # Get its four bird's-eye view coordinates. 
            point_squence = torch.stack([corners[:, 0], corners[:, 1], corners[:, 3], corners[:, 2]])
            xs = point_squence.T[0] * 10 + 400
            ys = -point_squence.T[1] * 10 + 400

            # Grab the current bounding box. 
            current_bbs.append((xs, ys))

            # Find the bin/grid cell it falls in, get its class mapping. 
            center_x, center_y = torch.mean(xs).item(), torch.mean(ys).item()
            key = (round_down(center_x), round_down(center_y))
            if key not in class_dict:
                print(key)
            bin_id = class_dict[key]
            bins[bin_id] = 1
            count += 1
            
        
        counts[count] = 1

        # Label Smoothing #
        if count > 10 and count < 88:
            counts[count+1] = 0.2
            counts[count-1] = 0.2
        target_counts.append(torch.Tensor(counts))
        
        images.append(torch.stack(six_images))
                
        target.append(torch.Tensor(bins))
        
        bbs.append(current_bbs)
                
    boom = torch.stack(images), torch.stack(target), torch.stack(road_maps), bbs, torch.stack(target_counts), torch.stack(road_bins)
    return boom

In [9]:
# The labeled dataset can only be retrieved by sample.
# And all the returned data are tuple of tensors, since bounding boxes may have different size
# You can choose whether the loader returns the extra_info. It is optional. You don't have to use it.
val_transform = transforms.ToTensor()

train_transform = transforms.Compose([
    transforms.RandomApply([
        transforms.ColorJitter(brightness = 0.5, contrast = 0.5, saturation = 0.4, hue = (-0.5, 0.5)),
        transforms.Grayscale(3),
#         transforms.RandomAffine(3),
    ]),
    transforms.ToTensor(),
])


labeled_trainset = LabeledDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=train_labeled_scene_index,
                                  transform=train_transform,
                                  extra_info=True
                                 )
labeled_valset = LabeledDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=val_labeled_scene_index,
                                  transform=val_transform,
                                  extra_info=True
                                 )

train_loader = torch.utils.data.DataLoader(labeled_trainset, batch_size=16, num_workers=10, shuffle=True, collate_fn=collate_fn)
val_loader = torch.utils.data.DataLoader(labeled_valset, batch_size=16, num_workers=2, shuffle=True, collate_fn=collate_fn)



#### Testing

In [6]:
sample, target, road_img, bbs, counts = iter(train_loader).next()

ValueError: too many values to unpack (expected 5)

In [ ]:
idx = -1

In [ ]:
idx += 1

In [ ]:
print(torch.argmax(counts[idx]))

In [ ]:
plt.imshow(torchvision.utils.make_grid(sample[idx], nrow=3).numpy().transpose(1, 2, 0))
plt.axis('off');

In [ ]:
fig, ax = plt.subplots()
ax.imshow(road_img[idx], cmap ='binary');
ax.plot(400, 400, 'x', color="red")

# `target` is 32 by 81. Find the indices where there's a 1. 
bin_ids = (target[idx] == 1).nonzero()
for bin_id in bin_ids:
    class_box = reverse_class_dict[bin_id]
    
    draw_box(ax, class_box, 'green')
    
def append_first_to_last(tens):
    ret = torch.cat((tens, torch.as_tensor([tens[0]])))
    return ret

    
for bb in bbs[idx]:
    ax.plot(append_first_to_last(bb[0]), append_first_to_last(bb[1]), color='orange')
    


In [ ]:
def draw_box(ax, class_box, color):
    box_xs = [class_box[0], class_box[0], class_box[0]+50, class_box[0]+50, class_box[0]]
    box_ys = [class_box[1], class_box[1]+50, class_box[1]+50, class_box[1], class_box[1]]
    ax.plot(box_xs, box_ys, color=color)

### model

In [10]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        
        self.encoder = torchvision.models.resnet50()
        self.encoder.fc = nn.Identity()
        self.concat_dim = 200 * 6
        
        self.compress = nn.Sequential(OrderedDict([
            ('linear0', nn.Linear(2048, 200)),
            ('drop', nn.Dropout(p = 0.5)),
            ('relu', nn.ReLU()),
        ]))
        
        self.classification = nn.Sequential(OrderedDict([
            ('linear1', nn.Linear(self.concat_dim, 256)),
        ]))
        
        self.counts = nn.Sequential(OrderedDict([
            ('count1', nn.Linear(self.concat_dim, 90))
        ]))
        
        self.segmentation = nn.Sequential(OrderedDict([
            ('linear1_segmentation', nn.Linear(self.concat_dim, 6400)),
            ('sigmoid', nn.Sigmoid())
        ]))
        
    def forward(self, x):
        batch_size = x.shape[0]
        num_images = x.shape[1]
        channels = x.shape[2]
        height = x.shape[3]
        width = x.shape[4]
        # Reshape here
        x = x.view(-1, channels, height, width)
        x = self.encoder(x)
        x = self.compress(x)
        x = x.view(-1, self.concat_dim)
        return self.classification(x), self.counts(x), self.segmentation(x)

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleModel()

# Weighting certain classes more. 
# positive_weight = torch.ones(256).to(device)
# for (x, y), bin_id in class_dict.items():
#     if abs(x - 400) <= 200 and abs(y - 400) <= 200:
#         positive_weight[bin_id] = 2

# for name, param in model.encoder.named_parameters():
#     if("bn" not in name):
#         param.requires_grad = False
        
# unfreeze_layers = [model.encoder.layer3, model.encoder.layer4]
# for layer in unfreeze_layers:
#     for param in layer.parameters():
#         param.requires_grad = True
        
model = model.to(device)
bin_criterion = nn.BCEWithLogitsLoss()
count_criterion = nn.BCEWithLogitsLoss()
segmentation_criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)
best_val_loss = 100

In [12]:
def mixup_data(x, y, alpha=0.2, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).to(device)
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a.float()) + (1 - lam) * criterion(pred, y_b.float())

In [15]:
def train():
    model.train()
    labeled_trainset = LabeledDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=train_labeled_scene_index,
                                  transform=train_transform,
                                  extra_info=True
                                 )
    train_loader = torch.utils.data.DataLoader(labeled_trainset, batch_size=10, num_workers=10, shuffle=True, collate_fn=collate_fn)
    
    train_losses = []
    bin_losses = []
    count_losses = []
    segmentation_losses = []
    for i, (sample, target, road_img, bbs, target_count, road_bins) in enumerate(train_loader):
        optimizer.zero_grad()

        sample = sample.to(device)
        target = target.to(device)
        target_count = target_count.to(device)
        road_bins = road_bins.to(device)
        
#         sample, target_a, target_b, lam = mixup_data(sample, target)
#         sample, target_a, target_b = map(torch.autograd.Variable, (sample, target_a, target_b))
        
    # Why were you doing this?
#         batch_yhat = []
#         batch_ycount = []
#         for j, x in enumerate(sample):
#             y_hat, y_count, segmentation = model(x)
#             batch_yhat.append(y_hat)
#             batch_ycount.append(y_count)
        
#         y_hat = torch.stack(batch_yhat).squeeze()
#         y_count = torch.stack(batch_ycount).squeeze()
        
    
        y_hat, y_count, segmentation = model(sample)
        
        # Mixup criterion here
#         bin_loss = mixup_criterion(bin_criterion, y_hat, target_a, target_b, lam)
        
        bin_loss = bin_criterion(y_hat, target.float())
        count_loss = count_criterion(y_count, target_count.float())
        segmentation_loss = segmentation_criterion(segmentation, road_bins.float())
        loss = bin_loss + 2 * count_loss + segmentation_loss
        
        train_losses.append(loss.item())
        bin_losses.append(bin_loss.item())
        count_losses.append(count_loss.item())
        segmentation_losses.append(segmentation_loss.item())

        loss.backward()
        optimizer.step()

        if i % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(sample), len(train_loader.dataset),
                50. * i / len(train_loader), loss.item()))
            
    print("\nAverage Train Epoch Loss: ", np.mean(train_losses))
    print("Average Train Bin Epoch Loss: ", np.mean(bin_losses))
    print("Average Train Count Epoch Loss: ", np.mean(count_losses))
    print("Average Train Segmentation Epoch Loss: ", np.mean(segmentation_losses))
            
def val():
    model.eval()
    val_losses = []
    bin_losses = []
    count_losses = []
    segmentation_losses = []
    count_correct = 0
    count_off_by_1 = 0
    total_count = 0
    bin_correct = 0
    total_bins = 0
    for i, (sample, target, road_img, bbs, target_count, road_bins) in enumerate(val_loader):

        model.eval()

        sample = sample.to(device)
        target = target.to(device)
        road_bins = road_bins.to(device)
        target_count = target_count.to(device)

        with torch.no_grad():
#             batch_yhat = []
#             batch_ycount = []
#             for j, x in enumerate(sample):
#                 y_hat, y_count = model(x)
#                 batch_yhat.append(y_hat)
#                 batch_ycount.append(y_count)

#             y_hat = torch.stack(batch_yhat).squeeze()
#             y_count = torch.stack(batch_ycount).squeeze()
            
#             for j, x in enumerate(y_count):
#                 pred_count = torch.argmax(y_count[j])
#                 t_count = torch.argmax(target_count[j])
#                 if pred_count == t_count:
#                     count_correct += 1
#                 elif abs(pred_count - t_count) == 1:
#                     count_off_by_1 += 1
                      
#                 pred_bins = torch.topk(y_hat[j], k = pred_count).indices
#                 t_bins = (target[j] == 1).nonzero()
#                 for b in pred_bins:
#                     if b in t_bins:
#                         bin_correct += 1
                    
#                 total_bins += len(t_bins)
            
            y_hat, y_count, segmentation = model(sample)

#             total_count += y_count.size(0)
            
            bin_loss = bin_criterion(y_hat, target.float())
            count_loss = count_criterion(y_count, target_count.float())
            segmentation_loss = segmentation_criterion(segmentation, road_bins.float())
            loss = bin_loss + count_loss + segmentation_loss

            val_losses.append(loss.item())
            bin_losses.append(bin_loss.item())
            count_losses.append(count_loss.item())
            segmentation_losses.append(segmentation_loss.item())
            
    print("Average Validation Epoch Loss: ", np.mean(val_losses))
    print("Average Validation Bin Epoch Loss: ", np.mean(bin_losses))
    print("Average Validation Count Epoch Loss: ", np.mean(count_losses))
    print("Average Validation Segmentation Epoch Loss: ", np.mean(segmentation_losses))
#     print("\tAverage Validation Count Accuracy: ", 100*count_correct/total_count)
#     print("\tAverage Validation Count-off-by-1 Accuracy: ", 100*count_off_by_1/total_count)
#     if total_bins != 0:
#         print("\tAverage Validation Bin Accuracy: ", 100*bin_correct/total_bins)
#     print("\n")
    global best_val_loss
    if np.mean(val_losses) < best_val_loss:
        best_val_loss = np.mean(val_losses)
        torch.save(model.state_dict(), '/scratch/brs426/all_six_images_classify_count_10_block_size_val_batch_16_better.pt')

In [16]:
epochs = 20
for epoch in range(epochs):
    train()
    val()

Train Epoch: 0 [0/3276 (0%)]	Loss: 0.266872
Train Epoch: 0 [500/3276 (8%)]	Loss: 0.241891
Train Epoch: 0 [1000/3276 (15%)]	Loss: 0.225863
Train Epoch: 0 [1500/3276 (23%)]	Loss: 0.311955
Train Epoch: 0 [2000/3276 (30%)]	Loss: 0.345373
Train Epoch: 0 [2500/3276 (38%)]	Loss: 0.330010
Train Epoch: 0 [3000/3276 (46%)]	Loss: 0.220535

Average Train Epoch Loss:  0.2822167823409162
Average Train Bin Epoch Loss:  0.11873043149073677
Average Train Count Epoch Loss:  0.02265249876583722
Average Train Segmentation Epoch Loss:  0.11818135354849624
Average Validation Epoch Loss:  0.48930929228663445
Average Validation Bin Epoch Loss:  0.2024839585646987
Average Validation Count Epoch Loss:  0.07396501279436052
Average Validation Segmentation Epoch Loss:  0.21286032907664776
Train Epoch: 1 [0/3276 (0%)]	Loss: 0.231639
Train Epoch: 1 [500/3276 (8%)]	Loss: 0.341567
Train Epoch: 1 [1000/3276 (15%)]	Loss: 0.285201
Train Epoch: 1 [1500/3276 (23%)]	Loss: 0.318972
Train Epoch: 1 [2000/3276 (30%)]	Loss: 0.19

Average Validation Epoch Loss:  0.49812611751258373
Average Validation Bin Epoch Loss:  0.20717901457101107
Average Validation Count Epoch Loss:  0.08153295773081481
Average Validation Segmentation Epoch Loss:  0.20941414684057236
Train Epoch: 11 [0/3276 (0%)]	Loss: 0.208156
Train Epoch: 11 [500/3276 (8%)]	Loss: 0.298085
Train Epoch: 11 [1000/3276 (15%)]	Loss: 0.234396
Train Epoch: 11 [1500/3276 (23%)]	Loss: 0.295111
Train Epoch: 11 [2000/3276 (30%)]	Loss: 0.324071
Train Epoch: 11 [2500/3276 (38%)]	Loss: 0.292374
Train Epoch: 11 [3000/3276 (46%)]	Loss: 0.325055

Average Train Epoch Loss:  0.26499676713492815
Average Train Bin Epoch Loss:  0.11377363209044788
Average Train Count Epoch Loss:  0.020064192974553783
Average Train Segmentation Epoch Loss:  0.11109474939634888
Average Validation Epoch Loss:  0.48207281343638897
Average Validation Bin Epoch Loss:  0.2023247480392456
Average Validation Count Epoch Loss:  0.07387874973937869
Average Validation Segmentation Epoch Loss:  0.2058693

In [13]:
# 60 x 200
# 10 x 1200
x = torch.randn((12, 5))

## Leaderboard submissions

In [17]:
# Compute threat scores
def reconstruct_from_bins(bins, block_size, threshold):
    print(bins.shape)
    road_map = torch.zeros((800, 800))
    idx = 0
    for x in range(0, 800, block_size):
        for y in range(0, 800, block_size):
            road_map[x:x+block_size, y:y+block_size] = bins[idx]
            idx += 1
    return road_map > threshold

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleModel().to(device)
model.load_state_dict(torch.load('/scratch/brs426/all_six_images_classify_count_10_block_size.pt'))

<All keys matched successfully>

In [ ]:
model.eval()

%matplotlib inline
threat_scores = 0
threshold = 0.42


with torch.no_grad():
    for i, (sample, target, road_img, bbs, target_count, road_bins) in enumerate(val_loader):
        sample = sample.to(device)
        target = target.to(device)
        road_bins = road_bins.to(device)
        target_count = target_count.to(device)
        
        y_hat, y_count, segmentation = model(sample)
        segmentation = segmentation.squeeze()
        road_img = road_img.squeeze()
        
        reconstructed_road_map = reconstruct_from_bins(segmentation, 10, threshold).cpu()
        fig, (ax1, ax2) = plt.subplots(1,2)
        ax1.imshow(road_img, cmap='binary')
        ax2.imshow(reconstructed_road_map, cmap='binary')
        ts_road_map = compute_ts_road_map(road_img,reconstructed_road_map)
        threat_scores += ts_road_map
        
    
    print("Average threat score", threat_scores / len(val_loader))   

torch.Size([6400])
torch.Size([6400])
torch.Size([6400])
torch.Size([6400])
torch.Size([6400])
torch.Size([6400])
torch.Size([6400])
torch.Size([6400])
torch.Size([6400])
torch.Size([6400])
torch.Size([6400])
torch.Size([6400])
torch.Size([6400])
torch.Size([6400])
torch.Size([6400])
torch.Size([6400])
torch.Size([6400])
torch.Size([6400])
torch.Size([6400])
torch.Size([6400])
torch.Size([6400])


<ipython-input-23-aa6cb9651633>:20: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, (ax1, ax2) = plt.subplots(1,2)


torch.Size([6400])
torch.Size([6400])
torch.Size([6400])
torch.Size([6400])
torch.Size([6400])
torch.Size([6400])
torch.Size([6400])
torch.Size([6400])
torch.Size([6400])
torch.Size([6400])
torch.Size([6400])
torch.Size([6400])
torch.Size([6400])


In [91]:

class_label = 0
class_dict = dict()
reverse_class_dict = []
for i in range(0, 800, 50):
    for j in range(0, 800, 50):
        class_dict[(i, j)] = class_label
        class_label += 1
        reverse_class_dict.append((i, j))


def get_bounding_boxes(self, samples):

    # samples is (batch_size, 6, 3, 256, 306)      

    bb_samples = []

    for x in samples:
        preds_class, preds_count = self.model.detection(x) 

        result = torch.topk(preds_class, k = torch.argmax(preds_count).item())
        pred_ids = result.indices

        bounding_boxes = []
        for idx in pred_ids[0]:
            buck_x, buck_y = reverse_class_dict[idx.item()]

            xs = torch.Tensor([buck_x, buck_x, buck_x + 50, buck_x + 50]).double()
            ys = torch.Tensor([buck_y+16, buck_y+36, buck_y+16, buck_y+36]).double()

            xs = xs - 400
            ys = 800 - ys # right-side up
            ys = ys - 400

            xs /= 10.
            ys /= 10.

            coords = torch.stack((xs, ys))
            bounding_boxes.append(coords)

        bounding_boxes = torch.stack(bounding_boxes).double().cuda()
        bb_samples.append(bounding_boxes)

    return tuple(bb_samples)


model.eval()

threat_scores = 0

with torch.no_grad():
    bb_samples = []
    for i, (sample, target, road_img, bbs, target_count, road_bins) in enumerate(val_loader):
        sample = sample.to(device)
        target = target.to(device)
        road_bins = road_bins.to(device)
        target_count = target_count.to(device)
        
        y_hat, y_count, segmentation = model(sample)
        
        result = torch.topk(y_hat, k = torch.argmax(y_count).item())
        pred_ids = result.indices

        bounding_boxes = []
        for idx in pred_ids[0]:
            buck_x, buck_y = reverse_class_dict[idx.item()]

            xs = torch.Tensor([buck_x, buck_x, buck_x + 50, buck_x + 50]).double()
            ys = torch.Tensor([buck_y+16, buck_y+36, buck_y+16, buck_y+36]).double()

            xs = xs - 400
            ys = 800 - ys # right-side up
            ys = ys - 400

            xs /= 10.
            ys /= 10.

            coords = torch.stack((xs, ys))
            bounding_boxes.append(coords)

        bounding_boxes = torch.stack(bounding_boxes).double().cuda()
        bb_samples.append(bounding_boxes)
        bb_samples = tuple(bb_samples)
        
        print(bbs)
        bb_samples = bb_samples[0].cpu()
        ts_bounding_box = compute_ats_bounding_boxes(bb_samples, bbs)
        
        threat_scores += ts_bounding_box
    
    print("Average threat score", threat_scores / len(val_loader))        

[[(tensor([720.7258, 720.4731, 674.0170, 674.2697], dtype=torch.float64), tensor([284.0238, 304.5063, 303.9379, 283.4554], dtype=torch.float64)), (tensor([348.8326, 348.8452, 394.3148, 394.3023], dtype=torch.float64), tensor([248.6888, 230.6341, 230.6614, 248.7162], dtype=torch.float64)), (tensor([754.6695, 754.4052, 707.3991, 707.6634], dtype=torch.float64), tensor([319.4501, 340.8722, 340.2971, 318.8749], dtype=torch.float64)), (tensor([363.8959, 364.1186, 409.5848, 409.3621], dtype=torch.float64), tensor([212.2098, 194.1564, 194.7127, 212.7661], dtype=torch.float64)), (tensor([64.8990, 65.0014, 17.0024, 16.9000], dtype=torch.float64), tensor([316.6336, 336.6375, 336.8878, 316.8838], dtype=torch.float64)), (tensor([573.0020, 573.1035, 620.2725, 620.1709], dtype=torch.float64), tensor([182.1661, 163.1918, 163.4397, 182.4139], dtype=torch.float64)), (tensor([98.3245, 98.0880, 54.1317, 54.3682], dtype=torch.float64), tensor([387.1317, 406.3047, 405.7669, 386.5939], dtype=torch.float64))

AttributeError: 'list' object has no attribute 'size'

In [ ]:
# 0.253 val bin loss

# Random Affine 3 degrees
# 0.263 val bin loss


# Need to do 5 * bin_loss + count_loss or something like that. Also more extreme Random Affine maybe?

# Random Affine 5 degrees
# 0.266 val bin loss

# Took out Random Affine. 
# 0.268 val bin loss

# Increased compress dim from 128 to 200. 
# 0.259 val bin loss

# 5 * bin_loss + count_loss
# 0.249 + 0.055

# 8 *
# 0.251 + 0.054

# 8*, RandomAffine(3)
# 0.255

# 8*, RandomAffine(3), weight_decay 0.1

# 10 *, RandomAffine(3)
# 0.259

# 8 *, Normalize (mean, std)
# 0.26

# 8 *, Dropout
# 0.241, 0.253

# 5 *, Dropout
# 0.254

# 11 *, Dropout
# 0.249

# Want to try positive-weights for classes within 200 to 600. 
# Want to get the model to get those classes correct. 

# Mixup 0.2, 1 *, Dropout
# (0.244, 0.053), 


## Testing Model Output

In [ ]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        
        self.encoder = torchvision.models.resnet50()
        self.encoder.fc = nn.Identity()
        self.concat_dim = 200 * 6
        
        self.compress = nn.Sequential(OrderedDict([
            ('linear0', nn.Linear(2048, 200)),
            ('drop', nn.Dropout(p = 0.5)),
            ('relu', nn.ReLU()),
        ]))
        
        self.classification = nn.Sequential(OrderedDict([
            ('linear1', nn.Linear(self.concat_dim, 256)),
        ]))
        
        self.counts = nn.Sequential(OrderedDict([
            ('count1', nn.Linear(self.concat_dim, 90))
        ]))
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.compress(x)
        x = x.view(-1, self.concat_dim)
        return self.classification(x), self.counts(x)

In [ ]:
model = SimpleModel()
model.load_state_dict(torch.load('all_six_images_classify_count.pt'))
model.cuda()
model.eval()

In [ ]:
class_label = 0
class_dict = dict()
reverse_class_dict = []
for i in range(0, 800, 50):
    for j in range(0, 800, 50):
        class_dict[(i, j)] = class_label
        class_label += 1
        reverse_class_dict.append((i, j))

In [ ]:
def get_bounding_boxes(samples):
    
    # samples is (batch_size, 6, 3, 256, 306)
    
    # You need to return a tuple with size batch_size and each element is a cuda tensor [N, 2, 4]
    # where N is the number of bounding boxes. 
    
    # Okay so I have my model. 
    # 
    
    bb_samples = []
    
    for x in samples:
        preds_class, preds_count = model(x)
        
        # preds class is a 256-dimensional tensor, filled with probabilities
        # I need to find the `preds_count` top indices with the top values.
        
        
        result = torch.topk(preds_class, k = torch.argmax(preds_count).item())
        pred_ids = result.indices
        
        bounding_boxes = []
        for idx in pred_ids[0]:
            buck_x, buck_y = reverse_class_dict[idx.item()]
            
            xs = torch.as_tensor([buck_x, buck_x, buck_x + 50, buck_x + 50])
            ys = torch.as_tensor([buck_y+16, buck_y+36, buck_y+16, buck_y+36])
            
            xs = xs - 400
            ys = 800 - ys # right-side up
            ys = ys - 400
            
            xs /= 10
            ys /= 10
               
            coords = torch.stack((xs, ys))
            bounding_boxes.append(coords)
            
        bounding_boxes = torch.stack(bounding_boxes).cuda()
        bb_samples.append(bounding_boxes)
    
    return tuple(bb_samples)
    

In [ ]:
sample, target, road_img, bbs, counts = iter(val_loader).next()

In [ ]:
sample = sample.cuda()

In [ ]:
boom = get_bounding_boxes(sample)

In [ ]:
sigmoid_preds = torch.sigmoid(model(sample[idx])[0]).squeeze()

In [ ]:
idx = -1

In [ ]:
idx += 1

In [ ]:
plt.imshow(torchvision.utils.make_grid(sample[idx].cpu().detach(), nrow=3).numpy().transpose(1, 2, 0))
plt.axis('off');

In [ ]:
fig, ax = plt.subplots()
ax.imshow(road_img[idx], cmap ='binary');
ax.plot(400, 400, 'x', color="red")

# `target` is 32 by 81. Find the indices where there's a 1. 

bin_ids = (sigmoid_preds > 0.25).nonzero()
for bin_id in bin_ids:
    class_box = reverse_class_dict[bin_id]
    draw_vish_box(ax, class_box, 'red')
    
bin_ids = (target[idx] == 1).nonzero()
for bin_id in bin_ids:
    class_box = reverse_class_dict[bin_id]
    draw_vish_box(ax, class_box, 'green')

    
for bb in boom[idx]:
    box = bb.cpu().detach()
    draw_box(ax, box, 'orange')

In [ ]:
torch.stack([box[:, 0], box[:, 1], box[:, 3], box[:, 2], box[:, 0]])

def draw_box(ax, corners, color):
    point_squence = torch.stack([corners[:, 0], corners[:, 1], corners[:, 3], corners[:, 2], corners[:, 0]])
    
    # the corners are in meter and time 10 will convert them in pixels
    # Add 400, since the center of the image is at pixel (400, 400)
    # The negative sign is because the y axis is reversed for matplotlib
    ax.plot(point_squence.T[0] * 10 + 400, -point_squence.T[1] * 10 + 400, color=color)
    return point_squence.T[0] * 10 + 400, -point_squence.T[1] * 10 + 400

In [ ]:
def draw_vish_box(ax, class_box, color):
    box_xs = [class_box[0], class_box[0], class_box[0]+50, class_box[0]+50, class_box[0]]
    box_ys = [class_box[1], class_box[1]+50, class_box[1]+50, class_box[1], class_box[1]]
    ax.plot(box_xs, box_ys, color=color)